In [495]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import pandas as pd
import mne
data_dir = 'data_meg'
subj = "R2490"
dataqual = 'prepro' #or loc/exp
exp = 'loc' #or exp
dtype = "raw"
label_dir = 'data_log'
save_dir = 'data_meg'
bad_channels_dict = {
    "R2490": ['MEG 014', 'MEG 004', 'MEG 079', 'MEG 072', 'MEG 070', 'MEG 080', 'MEG 074', 'MEG 067', 'MEG 082', 'MEG 105', 'MEG 115', 'MEG 141', 'MEG 153'],
    "R2488": ['MEG 015', 'MEG 014', 'MEG 068', 'MEG 079', 'MEG 146', 'MEG 147', 'MEG 007', 'MEG 141'],
    "R2487": ['MEG 015', 'MEG 014', 'MEG 068', 'MEG 079', 'MEG 147', 'MEG 146', 'MEG 004'],
    "R2280": ['MEG 015', 'MEG 039', 'MEG 077', 'MEG 076', 'MEG 073', 'MEG 079', 'MEG 064', 'MEG 059', 'MEG 070']
}
bad_channels = bad_channels_dict.get(subj, [])

Fro R2488, exclude first event (which is practice)
For 

In [496]:
# raw = mne.io.read_raw_fif('data_meg/R2490/prepro/R2490_exp.fif', preload='temp_raw.fif')
raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)
raw.info['bads'].extend(bad_channels)
sfreq = raw.info['sfreq']
raw.filter(1, 40, method='iir')
downsample = 10
raw.resample(sfreq / downsample)

Opening raw data file data_meg/R2490/prepro/R2490_loc.fif...


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_64559/1403648187.py:2: RuntimeWarning: This filename (data_meg/R2490/prepro/R2490_loc.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)


    Range : 0 ... 669999 =      0.000 ...   669.999 secs
Ready.
Reading 0 ... 669999  =      0.000 ...   669.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

425 events found on stim channel STI 014
Event IDs: [160 164 165 166]
362 events found on stim channel STI 014
Event IDs: [160 164 165 166]


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_64559/1403648187.py:7: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw.resample(sfreq / downsample)


<Raw | R2490_loc.fif, 193 x 67000 (670.0 s), ~100.2 MB, data loaded>

In [497]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 160,
    'move': 161,
    'reveal_red': 162,
    'reveal_white': 163,
    'done': 164,
    'choice': 165,
    'timeout': 166
}

# Define trials to remove
trials_to_remove = []

362 events found on stim channel STI 014
Event IDs: [160 164 165 166]


In [498]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']] 
timeout_events = events[events[:, 2] == event_id['timeout']]
choice_events = events[events[:, 2] == event_id['choice']]
sfreq = raw.info['sfreq']

In [499]:
if subj == "R2488" :
    done_events = done_events[1:]

In [500]:

# Initialize a list to store filtered done events
filtered_done_events = [done_events[0]]  # Start with the first event

# Check for at least 3 seconds between each done event
for i in range(1, len(done_events)):
    time_diff = (done_events[i, 0] - done_events[i-1, 0]) / sfreq
    if time_diff < 2:
        print(f"Warning: Less than 3 seconds between done events at indices {i-1} and {i}")
    
    else:
        filtered_done_events.append(done_events[i])

# Use filtered_done_events for further processing
print(f"Filtered done events count: {len(filtered_done_events)}")



Filtered done events count: 120


In [501]:
choice_events = events[events[:, 2] == event_id['choice']]
# start_events = np.array([[done_event[0] - int(3.4 * sfreq), 0, event_id['start']] for done_event in done_events])
start_events = events[events[:, 2] == event_id['start']]
start_events = start_events[-120:]
sfreq = raw.info['sfreq']  # Sampling frequency
locolizer = pd.read_csv(f'{label_dir}/{subj}/loc_data.csv')

# Initialize a list to store trial information
trial_info = []
start_idx = 0

# Iterate through each reconstructed start event to create trial information
for start_event, done_event in zip(start_events, done_events):
    start_sample = start_event[0]
    # done_sample = done_event[0]
    done_sample = start_sample + int(3.4 * sfreq)
    # Calculate tmin and tmax for the epoch
    tmin = -0.2  # 0.2 s before 'start'
    tmax = (done_sample - start_sample) / sfreq  # Duration from 'start' to 'done'
    # Find choice events within the trial
    choice_event = choice_events[(choice_events[:, 0] > start_sample) & 
                                 (choice_events[:, 0] < done_sample + int(0.5*sfreq))]
    choice_time = choice_event[-1, 0] if len(choice_event) > 0 else None
    
    # Store trial information
    trial_info.append({
        'event_sample': start_sample,
        'trial_index': start_idx,
        'duration': tmax,
        'tmin': tmin,
        'tmax': tmax,
        'done': len(done_events) > 0,
        'done_times': done_sample / sfreq,
        'start_times': start_sample / sfreq,
        'choice_event': len(choice_event) > 0,
        'choice_time': choice_time / sfreq if len(choice_event) > 0 else None
    })
    start_idx += 1

In [502]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
choice_events = events[events[:, 2] == event_id['choice']]
new_events = np.array([[info['event_sample'], 0, event_id['start']] for info in trial_info])
tmin = -0.2  
# Initialize lists to store individual epochs data and trial information
data_list = []

for idx, event in enumerate(new_events):
    event_id_code = event[2]
    choice_time = trial_info[idx]['choice_time']  # Retrieve choice_time for the current trial
    print(choice_time)
    if choice_time is not None:
        start_sample = int(choice_time * sfreq)  # Convert choice_time to sample index
        tmin = -0.5  # 0.5 s before 'choice'
        tmax = 0.5  # 0.5 s after 'choice'
    else:
        done_sample = int(trial_info[idx]['done_times'] * sfreq)  # Convert done_times to sample index
        tmin = -1  # 1 s before 'done'
        tmax = 0  # At 'done'
    picks = mne.pick_types(raw.info, meg=True, exclude='bads')

    epochs = mne.Epochs(
        raw, [event], event_id={f'event_{event_id_code}': event_id_code},
        tmin=tmin, tmax=tmax, preload=True,picks=picks,
        reject_by_annotation=False, reject=None, verbose=True
    )
    data_list.append(epochs.get_data())

42.06
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
46.45
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
52.03
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
56.3
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101

120